In [8]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from bs4 import BeautifulSoup
import requests
import csv
import os

In [9]:
import sys
sys.path.insert(0, '/aclImdb/')

In [10]:
os.getcwd()

'/Users/Stephen/Documents/BeastMode'

In [11]:
# get all neg urls
with open('aclImdb/train/urls_pos.txt', 'r') as f:
    train_pos_urls = f.readlines()
    print len(train_pos_urls)

12500


In [12]:
def get_movie(url):
    pageText = requests.get(url)
    while (pageText == None):
        time.sleep(10)
        pageText = requests.get(url)
    soup = BeautifulSoup(pageText.text,"html.parser")
    return soup.find("div",attrs={"id":"tn15title"}).find("a").get_text()

In [46]:
% time
import random
# Make a dictionary of URL: movie name
url_dict = dict(zip(train_pos_urls, [None]*len(train_pos_urls)))
i = 0
for url in train_pos_urls[5275:]:
    if url_dict[url] == None:
        url_dict[url] = get_movie(url)
    if random.random() < 0.01:
        print i
    i += 1
    time.sleep(0.001)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.87 µs
88
188
289
441
517
585
693
725
853
917
946
975
1012
1051
1182
1308
1483
1529
1700
1734
1876
1883
1982
2201
2247
2282
2425
2544
2962
2968
3066
3070
3167
3188
3222
3259
3441
3492
3551
3559
3815
3853
3889
3962
3982
3989
4004
4012
4191
4276
4340
4515
4655
4919
5003
5018
5150
5163
5252
5296
5502
5521
5564
5800
5855
5983
5997
6547
6668
6682
6696
6760
6844
7033
7069
7111


If we did this right, the number of keys in `url_dict` should be equal to the number of unique urls in `train_pos_urls`. 

In [51]:
n_url_keys = len(url_dict.keys())
n_unique_urls = len(list(set(train_pos_urls)))
print n_url_keys, n_unique_urls

1390 1390


Let's save the results in a JSON file.

In [53]:
import json
fp = open("url_movie_tr_pos.json","w")
json.dump(url_dict, fp)
fp.close()

In [ ]:
# Make a review data frame
review_df = pd.DataFrame(columns=['movie_id', 'stars', 'positive', 'text', 'url', 'movie_name'])

In [ ]:
train_pos_names = list(os.walk('aclImdb/train/pos/'))[0][2]
for review in train_pos_names:
    stars = int(review.split("_")[1].split(".")[0])
    movieID = int(review.split("_")[0])
    fp = open('aclImdb/train/pos/%(review)s' % {'review': review}, 'r')
    text = fp.read()
    pos = True
    url = train_pos_urls[movieID]
    movie_name = url_dict[url]
    reviewDict = {'movie_id': movieID, 'stars': stars, 'positive': pos, 'text': text, 'url': url, 'movie_name': movie_name}
    review_df = review_df.append(pd.DataFrame(reviewDict),index=[0])
        